Перед началом запуска загрузите файл(ы), из который собираем Базу знания для Илоны

In [19]:
### Установка фреймворков
# !pip install -U langchain (общий фреймворк)
# !pip install langchain_community (для работы с загрузчиком документов)
!pip install -U sentence-transformers
# !pip install -U langchain-chroma (для создание БД)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.4 MB/s eta 0:00:00


In [34]:
# Загрузчик документов
from langchain.document_loaders import DirectoryLoader, TextLoader

# в loader указываем путь в папку с базой знаний
# path - укажите путь к ПАПКЕ с файлами для Базы знаний
# glob - файлы какого формата будут учитываться

loader = DirectoryLoader(path='/content', glob = './*.txt', loader_cls= TextLoader)
documents = loader.load()

In [30]:
# Разбинеие на чанки
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)
documents = text_splitter.split_documents(documents)

In [31]:
# Эмбеддинг
from langchain.embeddings import HuggingFaceBgeEmbeddings

# use HuggingFace embedding model
model_name = 'BAAI/bge-small-en-v1.5'
encode_kwargs = {'normalize_embeddings': True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    #use gpu for faster embedding creation
    encode_kwargs=encode_kwargs
)

In [32]:
# Создание БД
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(documents, bge_embeddings)

In [33]:
#  Ретривер
retriever = vectordb.as_retriever()
result = retriever.get_relevant_documents("where is human?")
print(result[0].page_content)